# Lesson 4: Tool Use and Conversational Chess

## Setup

In [7]:
# !pip install autogen openai
# !pip install chess

In [1]:
# reset all those keys
import os
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
llm_config = {"model": "gpt-4-turbo"}

In [8]:
import chess
import chess.svg
from typing_extensions import Annotated

## Initialize the chess board

In [9]:
board = chess.Board()

In [10]:
made_move = False

## Define the needed tools

### 1. Tool for getting legal moves

In [49]:
def get_legal_moves(

) -> Annotated[str, "A list of legal moves in UCI format"]:
    return "Possible moves are: " + ",".join(
        [str(move) for move in board.legal_moves]
    )
"""
retrieves the list of legal chess moves based on the current state of the chess board.
It is linked to the board_proxy, a proxy for interacting with the chess board, that for some reason, does not show here.
"""

'\nretrieves the list of legal chess moves based on the current state of the chess board. \nIt is linked to the board_proxy, a proxy for interacting with the chess board, that for some reason, does not show here.\n'

### 2. Tool for making a move on the board

In [12]:
def make_move(
    move: Annotated[str, "A move in UCI format."]
) -> Annotated[str, "Result of the move."]:
    move = chess.Move.from_uci(move)
    board.push_uci(str(move))
    global made_move
    made_move = True

    # Display the board.
    display(
        chess.svg.board(
            board,
            arrows=[(move.from_square, move.to_square)],
            fill={move.from_square: "gray"},
            size=200
        )
    )

    # Get the piece name.
    piece = board.piece_at(move.to_square)
    piece_symbol = piece.unicode_symbol()
    piece_name = (
        chess.piece_name(piece.piece_type).capitalize()
        if piece_symbol.isupper()
        else chess.piece_name(piece.piece_type)
    )
    return f"Moved {piece_name} ({piece_symbol}) from "\
    f"{chess.SQUARE_NAMES[move.from_square]} to "\
    f"{chess.SQUARE_NAMES[move.to_square]}."

## Create agents

You will create the player agents and a board proxy agents for the chess board.

In [18]:
# !pip install dask

In [13]:
from autogen import ConversableAgent

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [14]:
# Player white agent
player_white = ConversableAgent(
    name="Player White",
    system_message="You are a chess player and you play as white. "
    "First call get_legal_moves(), to get a list of legal moves. "
    "Then call make_move(move) to make a move.",
    llm_config=llm_config,
)

In [15]:
# Player black agent
player_black = ConversableAgent(
    name="Player Black",
    system_message="You are a chess player and you play as black. "
    "First call get_legal_moves(), to get a list of legal moves. "
    "Then call make_move(move) to make a move.",
    llm_config=llm_config,
)

In [16]:
def check_made_move(msg):
    global made_move
    if made_move:
        made_move = False
        return True
    else:
        return False


In [19]:
board_proxy = ConversableAgent(
    name="Board Proxy",
    llm_config=False,
    is_termination_msg=check_made_move,
    default_auto_reply="Please make a move.",
    human_input_mode="NEVER",
)

## Register the tools

A tool must be registered for the agent that calls the tool and the agent that executes the tool.

In [20]:
from autogen import register_function

In [21]:
for caller in [player_white, player_black]:
    register_function(
        get_legal_moves,
        caller=caller,
        executor=board_proxy,
        name="get_legal_moves",
        description="Get legal moves.",
    )

    register_function(
        make_move,
        caller=caller,
        executor=board_proxy,
        name="make_move",
        description="Call this tool to make a move.",
    )

/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2492: UserWarning: Function 'get_legal_moves' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)
/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2492: UserWarning: Function 'make_move' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


In [22]:
player_black.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Get legal moves.',
   'name': 'get_legal_moves',
   'parameters': {'type': 'object', 'properties': {}, 'required': []}}},
 {'type': 'function',
  'function': {'description': 'Call this tool to make a move.',
   'name': 'make_move',
   'parameters': {'type': 'object',
    'properties': {'move': {'type': 'string',
      'description': 'A move in UCI format.'}},
    'required': ['move']}}}]

## Register the nested chats

Each player agent will have a nested chat with the board proxy agent to
make moves on the chess board.

In [23]:
player_white.register_nested_chats(
    trigger=player_black,
    chat_queue=[
        {
            "sender": board_proxy,
            "recipient": player_white,
            "summary_method": "last_msg",
        }
    ],
)

player_black.register_nested_chats(
    trigger=player_white,
    chat_queue=[
        {
            "sender": board_proxy,
            "recipient": player_black,
            "summary_method": "last_msg",
        }
    ],
)

## Start the Game

The game will start with the first message.

<p style="background-color:#ECECEC; padding:15px; "> <b>Note:</b> In this lesson, you will use GPT 4 for better results. Please note that the lesson has a quota limit. If you want to explore the code in this lesson further, we recommend trying it locally with your own API key.

**Note**: You might get a slightly different moves than what's shown in the video.

In [40]:
# this is a pure chat thing, but the chess thing was the same idea.
# cathy = ConversableAgent(
#     name="cathy",
#     system_message=
#     "Your name is Cathy and you are a stand-up comedian.",
#     llm_config=llm_config,
#     human_input_mode="NEVER",
# )

# joe = ConversableAgent(
#     name="joe",
#     system_message=
#     "Your name is Joe and you are a stand-up comedian. "
#     "Start the next joke from the punchline of the previous joke.",
#     llm_config=llm_config,
#     human_input_mode="NEVER",
# )

# # agent conversation definition
# chat_result = joe.initiate_chat(
#     recipient=cathy,
#     message="I'm Joe. Cathy, let's keep the jokes rolling.",
#     max_turns=2,
# )

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, ready to laugh? Why don’t scientists trust atoms anymore? Because they make up everything! How's that for a warm-up? Let’s keep the giggles going!

--------------------------------------------------------------------------------
joe (to cathy):

That’s a good one, Cathy! Speaking of making stuff up, have you ever noticed how a kid’s explanation for why they're covered in mud somehow involves a dragon, a time machine, and a promise they definitely didn’t break? I'd trust an atom any day over a 6-year-old with a wild story and mysteriously missing cookies! So, what's your next brain-bender?

--------------------------------------------------------------------------------
cathy (to joe):

Right, Joe? Kids have imaginations bigger than their little muddy pockets! Okay, brace yourself for this next head-scratcher. Why did 

In [47]:
# source: https://cbarkinozer.medium.com/autogen-ile-yz-ajanik-tasar%C4%B1m-desenleri-985c95c1fffa
llm_config = {"model": "gpt-4-turbo"}
import chess
import chess.svg
from typing_extensions import Annotated
board = chess.Board()
made_move = False

In [42]:
def get_legal_moves(

) -> Annotated[str, "A list of legal moves in UCI format"]:
    return "Possible moves are: " + ",".join(
        [str(move) for move in board.legal_moves]
    )

In [43]:
def make_move(
    move: Annotated[str, "A move in UCI format."]
) -> Annotated[str, "Result of the move."]:
    move = chess.Move.from_uci(move)
    board.push_uci(str(move))
    global made_move
    made_move = True

    # Display the board.
    display(
        chess.svg.board(
            board,
            arrows=[(move.from_square, move.to_square)],
            fill={move.from_square: "gray"},
            size=200
        )
    )

    # Get the piece name.
    piece = board.piece_at(move.to_square)
    piece_symbol = piece.unicode_symbol()
    piece_name = (
        chess.piece_name(piece.piece_type).capitalize()
        if piece_symbol.isupper()
        else chess.piece_name(piece.piece_type)
    )
    return f"Moved {piece_name} ({piece_symbol}) from "\
    f"{chess.SQUARE_NAMES[move.from_square]} to "\
    f"{chess.SQUARE_NAMES[move.to_square]}."

In [44]:
from autogen import ConversableAgent

# Player white agent
player_white = ConversableAgent(
    name="Player White",
    system_message="You are a chess player and you play as white. "
    "First call get_legal_moves(), to get a list of legal moves. "
    "Then call make_move(move) to make a move.",
    llm_config=llm_config,
)

# Player black agent
player_black = ConversableAgent(
    name="Player Black",
    system_message="You are a chess player and you play as black. "
    "First call get_legal_moves(), to get a list of legal moves. "
    "Then call make_move(move) to make a move.",
    llm_config=llm_config,
)

def check_made_move(msg):
    global made_move
    if made_move:
        made_move = False
        return True
    else:
        return False

board_proxy = ConversableAgent(
    name="Board Proxy",
    llm_config=False, # no llm use
    is_termination_msg=check_made_move, # stop conversing when a move has been made
    default_auto_reply="Please make a move.",
    human_input_mode="NEVER",
)

In [37]:
# not working due to network error that I do not know what to do right now.
# it is really cool if the board shows

In [36]:
# chat_result = player_black.initiate_chat(
# player_white,
# message="Let’s play chess! Your move.",
# max_turns=2,
# )

In [39]:
# board = chess.Board()

# chat_result = player_black.initiate_chat(
#     player_white,
#     message="Let's play chess! Your move.",
#     max_turns=2,
# )

## Adding a fun chitchat to the game!

In [26]:
player_white = ConversableAgent(
    name="Player White",
    system_message="You are a chess player and you play as white. "
    "First call get_legal_moves(), to get a list of legal moves. "
    "Then call make_move(move) to make a move. "
    "After a move is made, chitchat to make the game fun.",
    llm_config=llm_config,
)

In [27]:
player_black = ConversableAgent(
    name="Player Black",
    system_message="You are a chess player and you play as black. "
    "First call get_legal_moves(), to get a list of legal moves. "
    "Then call make_move(move) to make a move. "
    "After a move is made, chitchat to make the game fun.",
    llm_config=llm_config,
)

In [28]:
for caller in [player_white, player_black]:
    register_function(
        get_legal_moves,
        caller=caller,
        executor=board_proxy,
        name="get_legal_moves",
        description="Get legal moves.",
    )

    register_function(
        make_move,
        caller=caller,
        executor=board_proxy,
        name="make_move",
        description="Call this tool to make a move.",
    )

player_white.register_nested_chats(
    trigger=player_black,
    chat_queue=[
        {
            "sender": board_proxy,
            "recipient": player_white,
            "summary_method": "last_msg",
            "silent": True,
        }
    ],
)

player_black.register_nested_chats(
    trigger=player_white,
    chat_queue=[
        {
            "sender": board_proxy,
            "recipient": player_black,
            "summary_method": "last_msg",
            "silent": True,
        }
    ],
)

In [33]:
# board = chess.Board()

# chat_result = player_black.initiate_chat(
#     player_white,
#     message="Let's play chess! Your move.",
#     max_turns=2,
# )

**Note:**
To add human input to this game, add **human_input_mode="ALWAYS"** for both player agents.